In [ ]:
import datetime
import math
import numpy as np
import time
import xml.etree.ElementTree as ET

In [ ]:

class Coordinate:
    def __init__(self, lon: float, lat: float, elevation: float):
        self.lon = lon
        self.lat = lat
        self.elevation = elevation

    def __str__(self):
        return f"Lat: {self.lat} | Lon: {self.lon} | Elevation: {self.elevation}"
    
    def __repr__(self):
        return f"Lat: {self.lat} | Lon: {self.lon} | Elevation: {self.elevation}"
    
class Placemark:
    def __init__(self, name: str, coords: list[Coordinate]):
        self.name = name
        self.coords = coords

    def __str__(self):
        return f"{self.name}: {self.coords}"
    
    def __repr__(self):
        return f"{self.name}: {self.coords}"

In [ ]:

dayMs = 1000 * 60 * 60 * 24
J1970 = 2440588
J2000 = 2451545
rad = math.pi / 180.0
e = rad * 23.4397
cell_area = 0.0153 #m^2

def altitude(H, phi, dec):
    return math.asin(math.sin(phi) * math.sin(dec) + math.cos(phi) * math.cos(dec) * math.cos(H))

def azimuth(H, phi, dec):  
    return math.atan2(math.sin(H), math.cos(H) * math.sin(phi) - math.tan(dec) * math.cos(phi))

def siderealTime(d, lw):
     return rad * (280.16 + 360.9856235 * d) - lw

def rightAscension(l, b): 
    return math.atan2(math.sin(l) * math.cos(e) - math.tan(b) * math.sin(e), math.cos(l))

def declination(l, b):    
    return math.asin(math.sin(b) * math.cos(e) + math.cos(b) * math.sin(e) * math.sin(l))

def toJulian(date):
    return (time.mktime(date.timetuple()) * 1000) / dayMs - 0.5 + J1970

def toDays(date):   
    return toJulian(date) - J2000

def solarMeanAnomaly(d):
    return rad * (357.5291 + 0.98560028 * d)

def eclipticLongitude(M):
    C = rad * (1.9148 * math.sin(M) + 0.02 * math.sin(2 * M) + 0.0003 * math.sin(3 * M)) # equation of center
    P = rad * 102.9372 # perihelion of the Earth
    return M + C + P + math.pi

def sunCoords(d):
    M = solarMeanAnomaly(d)
    L = eclipticLongitude(M)
    return dict(
        dec= declination(L, 0),
        ra= rightAscension(L, 0)
    )

def getSunPosition(date, lat, lng):
    """Returns positional attributes of the sun for the given time and location."""
    lw  = rad * -lng
    phi = rad * lat
    d   = toDays(date)

    c  = sunCoords(d)
    H  = siderealTime(d, lw) - c["ra"]
    # print("d", d, "c",c,"H",H,"phi", phi)
    return dict(
        azimuth=azimuth(H, phi, c["dec"]),
        altitude=altitude(H, phi, c["dec"])
    )


In [ ]:
modules = {
"hood_front": {
    "left_stairs": [33.38, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
    "left_center_3x4": [33.38, 33.38, 33.38, 27.64, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
    "right_center_3x4": [33.38, 33.38, 33.38, 27.64, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
    "right_stairs": [33.38, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44]
},"top_front": {
    "leftmost_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
    "leftmost_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
    "leftmost_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
    "leftcenter_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
    "leftcenter_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
    "leftcenter_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
    "rightcenter_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
    "rightcenter_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
    "rightcenter_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
    "rightmost_top_4x3": [28.52, 28.52, 28.52, 28.52, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
    "rightmost_bottom_4x3": [22.16, 22.16, 22.16, 22.16, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14]
},"top_back": {
    "leftmost_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
    "leftcenter_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
    "rightcenter_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
    "rightmost_4x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64, -3.64]
},"back": {
    "leftmost_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
    "leftcenter_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
    "rightcenter_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
    "rightmost_top_4x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
    "leftmost_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
    "leftcenter_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64], 
    "rightcenter_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
    "rightmost_bottom-4x4": [-4, -4, -4, -4, -5, -5, -5, -5, -6, -6, -6, -6]
}}

In [ ]:
#equation to calculate distance between coordinates 
def heversine_and_azimuth(lon1: float, lat1: float, lon2: float, lat2: float):
    R = 6371.0  #Radius of the Earth in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    azimuth = math.atan2(math.sin(dlon)*math.cos(lat2), math.cos(lat1)*math.sin(lat2)-math.sin(lat1)*math.cos(lat2)*math.cos(dlon))
    
    azimuth %= math.pi*2

    return distance, azimuth

In [ ]:
from sklearn.linear_model import LinearRegression


def cell_solar_irradiance(lat, lon, time, elevation, module_tilt, car_azimuth_angle):
    
    module_tilt %= 360
    car_azimuth_angle = math.degrees(car_azimuth_angle)
    print(car_azimuth_angle)

    sunPosition = getSunPosition(time, lat, lon)
#     print("sunPosition: ", sunPosition)
    sun_elevation_angle = math.degrees(sunPosition['altitude'])
    if sun_elevation_angle < 0:
        sun_elevation_angle = 360 + sun_elevation_angle
#     print("sun_elevation_angle (deg): ", sun_elevation_angle)
    sun_azimuth_angle = math.degrees(sunPosition['azimuth'])
    if sun_azimuth_angle < 0:
        sun_azimuth_angle = 360 + sun_azimuth_angle
#     print("sun_azimuth_angle (deg): ", sun_azimuth_angle)
    
    #https://www.pveducation.org/pvcdrom/properties-of-sunlight/air-mass#AMequation
    #s_incident measured in kW/m^2 should be retrieved empirically
    air_mass = 1/(math.cos(math.radians(90-sun_elevation_angle)) + 0.50572*(96.07995-(90-sun_elevation_angle))**-1.6364) 
#     print("air_mass: ", air_mass)
#     print("elevation (km): ", elevation)
    
    s_incident_diffuse = 1.1*1.353*((1-0.14*elevation)*0.7**air_mass**0.678 + 0.14*elevation)
    print("s_incident_diffuse (kW/m^2): ", s_incident_diffuse)
    
    s_cell_diffuse = s_incident_diffuse*(math.cos(math.radians(sun_elevation_angle))*math.sin(math.radians(module_tilt))*math.cos(math.radians(car_azimuth_angle - sun_azimuth_angle)) 
                     + math.sin(math.radians(sun_elevation_angle))*math.cos(math.radians(module_tilt)))
    print("s_cell_diffuse (kW/m^2): ", s_cell_diffuse)
#     print("s_cell (kW): ", s_cell_diffuse*cell_area)
    
    return s_cell_diffuse*cell_area

def section_solar_irradiance(lat, lon, time, elevation, module_tilts, car_azimuth_angle):
    total_sum = 0
    #store solar irradiance value of similar angles in a dictionary and use it to approximate?
    #definitely keep reference of same values to limit api calls
    for module in module_tilts.keys():
        module_sum = 0
        for cell_angle in module_tilts[module]:
            cell_irr = cell_solar_irradiance(lat, lon, time, elevation, cell_angle, car_azimuth_angle)
            print("cell_irr: ", cell_irr)
            module_sum += cell_irr
        print("module sum: ", module_sum)
        total_sum += module_sum
    return total_sum

def total_irradiance(lat, lon, time, elevation, modules, car_azimuth_angle):
    # iterating over all of the module sections, which are hood_front, top_front, etc.
    total = 0
    for module_section in modules.keys():
        total += section_solar_irradiance(lat, lon, time, elevation/1000, modules[module_section], car_azimuth_angle)
    return total

    #sum of all the modules on the car

def energy_captured_along_route(time_initial: datetime.time, velocity: float, route: list[Coordinate]):
    current_time = time_initial

    xData = np.array([200, 300, 400, 500, 600, 700, 800, 900, 1000]).reshape((-1,1))
    yData = np.array([54, 83, 112, 141, 170, 200, 230, 260, 290])
    irrToPowerModel = LinearRegression()
    irrToPowerModel.fit(xData, yData)
    total_energy = 0

    for i in range(len(route)-2):
        distance, azimuth = heversine_and_azimuth(route[i].lon, route[i].lat, route[i+1].lon, route[i+1].lat)
        delta_time_in_sec = distance/velocity # is this in seconds?
        current_time += datetime.timedelta(seconds=delta_time_in_sec)
        mid_point_lat = (route[i].lat + route[i+1].lat)/2
        mid_point_lon = (route[i].lat + route[i+1].lat)/2
        mid_point_elevation = (route[i].elevation + route[i+1].elevation)/2        
        total_irradiance_at_midpoint = total_irradiance(mid_point_lat, mid_point_lon, current_time, mid_point_elevation, modules, azimuth)

        total_power_along_segment = irrToPowerModel.predict(np.array(total_irradiance_at_midpoint).reshape(1, -1))
        total_energy += total_power_along_segment*delta_time_in_sec
    return total_energy
#    for len(route):
#        ans += total_irradiance
    #sum of all 

In [ ]:
def parse_kml_file(filename):
    root = ET.parse(filename).getroot()
    placemarks = []
    for child in root[0]:
        if child.tag == '{http://www.opengis.net/kml/2.2}Placemark':
            # name tag
            name = child.find('{http://www.opengis.net/kml/2.2}name').text
            coords = []
            # coordinates tag
            p_coords = child.find('{http://www.opengis.net/kml/2.2}LineString').find('{http://www.opengis.net/kml/2.2}coordinates')
            for line in p_coords.text.split("\n"):
                line = line.strip() 
                if (len(line) == 0):
                    continue

                parts = line.split(",")
                coords.append(Coordinate(float(parts[0]),float(parts[1]),float(parts[2])))

            placemarks.append(Placemark(name, coords))


    return placemarks

In [ ]:
placemarks = parse_kml_file("data/Main Route.kml")

for placemark in placemarks:
    print(energy_captured_along_route(datetime.datetime(2023, 8, 21, 16, 0, 0), 50, placemark.coords))

In [ ]:
def energy_used_along_route():
    

In [ ]:
VOLTAGE = 96 #V
def estimate_soc(q_initial: float, time_initial: datetime.time, velocity: float, route: list[Coordinate]):
    q_final = q_initial
    return q_final